In [11]:
import cv2
from tkinter import Tk
from tkinter.filedialog import askopenfilename

from IPython.core.display import Markdown

In [12]:
def select_and_load_image():
    Tk().withdraw()
    image_path = askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.tiff")])

    if not image_path:
        print("No file selected.")
        return None

    # Load the selected image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError(f"Failed to load image from path: {image_path}")

    return image, cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def display_image_cv(image, title="Selected Image"):
    image = cv2.resize(image, (800, 600))
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
image, image_grey = select_and_load_image()
display_image_cv(image, "Original image")
display_image_cv(image_grey, "Greyscale image")

### Importing YOLOv11n model for barcode detection and training

In [14]:
from ultralytics import YOLO
import torch

model = YOLO("yolo11s.pt")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

device(type='cuda', index=0)

In [15]:
from pathlib import Path

if not Path('barcode-scanner.pt').exists():
    save_path = Path('C:\\Stuff\\Scoala\\An III\\PI\\code-bar-qr-scanner\\save')
    save_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

    model.train(
        data="ZVZ-real/dataset/dataset.yaml",
        epochs=10,
        imgsz=921,
        batch=12,
        device=device,
        project=str(save_path),  # Root directory for saving
        name="code-bar-qr-scanner",
        verbose=True,
        cache=False)
else:
    print("Model already trained and saved as barcode-scanner.pt")

Model already trained and saved as barcode-scanner.pt


### Predicting how well the model works

In [24]:
model = YOLO("barcode-scanner.pt")
image, _ = select_and_load_image()

predict = model.predict(image, conf=0.6, iou=0.5, device=device)
res_plotted = predict[0].plot()
display_image_cv(res_plotted, 'Predicted image')


0: 544x928 2 EAN13s, 1 QRCode, 11.3ms
Speed: 3.8ms preprocess, 11.3ms inference, 1.6ms postprocess per image at shape (1, 3, 544, 928)
